# Seguridad por oscuridad: introducción al criptoanálisis

Varias editoriales de todo el mundo han publicado una línea de cuentos infantiles en forma de coleccionable por entregas. El producto de ambas es similar: venden un altavoz y unas figuras coleccionables, y el cuento asociado a una figura suena cuando se acerca la figura al altavoz.

En algunos de estos productos, los cuentos se guardan en una tarjeta SD en el altavoz. El cuento no está en la figura porque eso provocaría que las figuras fuesen demasiado caras. Las figuras tienen etiquetas NFC o RFID pasivas con un identificador, que desbloquean el cuento que está en el altavoz. Una de las editoriales, para evitar que se pueda acceder a los cuentos sin comprar la figura, ha decidido cifrar los archivos de audio...pero han confiado en la seguridad por oscuridad

¿Podremos desbloquear los cuentos que están en el altavoz sin tener que comprar las figuras coleccionables?

![](https://github.com/Juanvvc/crypto/blob/master/ejercicios/01/images/caperu.jpg?raw=1)

**Atención**: este producto y el protocolo que usa la editorial es real. Lo único que no es real es la contraseña que utiliza la editorial para proteger sus cuentos, pero se puede obtener tal y como se describe en este ejercicio.

Vamos a analizar el sistema de cifrado que se utiliza en los audiocuentos.

La clase HexView que está a continuación es simplemente una forma sencilla de mostrar el contenido de un archivo en hexadecimal en estos notebook. Podrías también ejecutar un hexdump en línea de comandos. No es importante que entiendas esta clase, simplemente ejecútala (MAYÚSCULAS+ENTER) porque la usaremos después

In [ ]:
import os

class HexViewer():
    def __init__(self, filename, bs=16, count=32):
        self.filename = filename
        self.bs = bs
        self.count = count
    def get_blocks(self, offset=0):
        assert self.filename
        with open(self.filename, "rb") as file:
            try:
                file.seek(offset, os.SEEK_SET)
                block = file.read(self.bs * self.count)
            except ValueError: # Empty offsetSpinbox
                return
        rows = [block[i:i + self.bs]
                for i in range(0, len(block), self.bs)]
        for row in rows:
            yield '{} - {}'.format(self.get_bytes(row), self.get_ascii(row))
    def get_bytes(self, row):
        output = " ".join(map(lambda b:"{:02X}".format(b), row))
        if len(row) < self.bs:
            output += " " * (self.bs - len(row)) * 3
        return output
    def get_ascii(self, row, not_printable='.'):
        output = []
        for char in row.decode('ascii', errors="replace"):
            if char in "\u2028\u2029\t\n\r\v\f\uFFFD":
                char = not_printable
            elif not 0x20 <= ord(char) <= 0xFF:
                char = not_printable
            output.append(char)
        return "".join(output)
    def print_blocks(self, offset=0):
        print('\n'.join(self.get_blocks(offset)))
        

## ¿Cómo es un archivo MP3 real?

Bien, en este enlace: https://github.com/Juanvvc/crypto/raw/master/ejercicios/01/test.mp3 puedes encontrar un archivo `test.mp3` de prueba, que nos servirá para explorar cómo es un archivo MP3 real.

(Nota: el archivo MP3 original se descargó de esta web: <https://file-examples.com/index.php/sample-audio-files/sample-mp3-download/>)

Que el archivo cifrado en la memoria del altavoz es un MP3 y no otro formato es una suposición razonable. Podría ser también WAV, OGG, o algún otro formato de audio. En este caso son MP3. Si no lo hubiesen sido, un adversario tardaría un poco más tratando de descubrir en qué formato está el audio, pero no demasiado más: no hay tantos formatos de audio legibles por un altavoz de bajo coste.

Vamos a ver los primeros 512 bytes (32 bloques de 16 bytes) del archivo para conocer el formato que tiene un archivo MP3 de audio:

In [ ]:
import urllib.request
if not os.path.exists('test.mp3'): urllib.request.urlretrieve('https://github.com/Juanvvc/crypto/raw/master/ejercicios/01/test.mp3', 'test.mp3')
HexViewer('test.mp3', bs=16, count=32).print_blocks()

Observa:

- El inicio de un archivo es conocido: `ID3`. Es muy común que los archivos empiecen con unas pocos letras que los identifican: DOCX, JPG, ZIP... tienen todos una cabera inicial que los identifica.
- Los archivos MP3 como este en ocasiones tienen "secciones de padding", que son secciones con muchos ceros

## ¿Cómo es el archivo de un cuento "protegido"?

Veamos ahora el archivo `01.enc`. Este archivo está cifrado utilizando el mismo método que los audiocuentos que se pueden encontrar en el quiosco. Lo puedes encontrar en este enlace https://github.com/Juanvvc/crypto/raw/master/ejercicios/01/test.mp3, aunque el código de más abajo lo descarga y muestra sin que tengas que hacerlo tú.

Observa:

- El archivo protegido no empieza como empezaba el archivo MP3 real
- En las partes que un MP3 tendría ceros, aquí aparece otra cadena

Vamos a suponer que este archivo es un MP3 cifrado. Entonces, **sabemos** que tiene que empezar con la cadena "ID3" porque todos los archivos MP3 empiezan con esa cadena. Así que tenemos que encontrar una clave que descifre el inicio a la cadeba "ID3". Además, sabemos que es muy posible que partes del archivo descifrado contengan largas secciones con "ceros". Esto de conocer al menos partes de un mensaje y aprovechar el conocimiento para descifrar partes del mismo se llama "ataque de texto en claro conocido" ( https://en.wikipedia.org/wiki/Known-plaintext_attack ) y se usa en algunos ataques a sistemas criptográficos

In [ ]:
if not os.path.exists('01.enc'): urllib.request.urlretrieve('https://github.com/Juanvvc/crypto/raw/master/ejercicios/01/01.enc', '01.enc')
HexViewer('01.enc', bs=16, count=32).print_blocks()

Pregunta:

- ¿Sabrías cómo se ha cifrado este archivo?
- ¿Puedes crear un código para descifrarlo?

Pistas:

- XOR rotativo:https://en.wikipedia.org/wiki/XOR_cipher
- https://www.mikrocontroller.net/topic/503014
- Recomendación: utiliza la librería de Python PyCryptodome, porque la usaremos en el resto del curso: https://pycryptodome.readthedocs.io/en/latest/

Desde línea de comandos podrías intentar algo así, pero la idea es que escribas tu propio código:
    
```
# pip install xortool ; hash xortool-xor
file test.mp3
hexdump -C -n 32 test.mp3 
hexdump -C 01.enc | less 
cat 01.enc |  xortool-xor -r "secret" -f - > 01.mp3
```

## Otros ejercicios

Para completar conocimientos, te recomiendo que hagas los ejercicios del "Introducción a Cryptohack" de <www.cryptohack.org>. Necesitarás tener conocimientos básicos de Python.